In [5]:
%run ../../Utils/yp_utils.py

# Initial setup

In [6]:
paper_pmid = 26681516
paper_name = 'worley_capaldi_2015' 

In [7]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [8]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [9]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Full Screen 20min 0.375M KCl')

In [10]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4706 x 13


In [11]:
original_data.head()

,Standard,Common,Raw Fam (Nsr1),Raw Joe (Pex6),Normalized Fam-Joe,Library Well*,Library Plate*,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,YBR032W,YBR032W,16.02,20.71,4.540771,B5,p42,NaN,NaN,NaN,NaN,NaN,NaN
1,YBR077C,SLM4,17.92,22.44,4.370771,E8,p42,NaN,NaN,NaN,NaN,NaN,NaN
2,YBR020W,GAL1,17.67,21.76,3.526792,H10,p50,NaN,* These plate numbers correspond to the plate ...,NaN,NaN,NaN,NaN
3,YPL086C,ELP3,17.85,21.12,3.286897,G9,p24,NaN,NaN,NaN,NaN,NaN,NaN
4,YDL210W,UGA4,17.25,20.78,3.271846,E2,p38,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
original_data['orf'] = original_data['Standard '].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [14]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Standard     Common  Raw Fam (Nsr1)  Raw Joe (Pex6)  \
index_input                                                       
1575         YLR287-A  YLR287-A           20.94            21.6   

             Normalized Fam-Joe Library Well* Library Plate*  Unnamed: 7  \
index_input                                                                
1575                   0.505535            H9            p22         NaN   

            Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  \
index_input                                                                 
1575               NaN         NaN          NaN          NaN          NaN   

                  orf  
index_input            
1575         YLR287-A  


In [16]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
data_cols = ['Raw Fam (Nsr1)','Raw Joe (Pex6)','Normalized Fam-Joe']

In [19]:
original_data = original_data[data_cols].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4617, 3)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [21888,21932, 21933]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,21888,21932,21933
data_type,value,value,value
orf,,,
YAL002W,21.82,21.89,-0.035846
YAL004W,19.19,20.77,1.474154
YAL005C,18.84,20.98,2.034154
YAL008W,21.55,21.52,-0.135846
YAL009W,21.19,20.48,-0.815846


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21888,21932,21933
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,21.82,21.89,-0.035846
1863,YAL004W,19.19,20.77,1.474154
4,YAL005C,18.84,20.98,2.034154
6,YAL008W,21.55,21.52,-0.135846
7,YAL009W,21.19,20.48,-0.815846


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21888,21932,21933,21888,21932,21933
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,21.82,21.89,-0.035846,0.718161,1.311585,-0.020810
1863,YAL004W,19.19,20.77,1.474154,-1.451069,0.026540,1.885307
4,YAL005C,18.84,20.98,2.034154,-1.739749,0.267486,2.592211
6,YAL008W,21.55,21.52,-0.135846,0.495465,0.887062,-0.147042
7,YAL009W,21.19,20.48,-0.815846,0.198536,-0.306195,-1.005426


# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 26681516...
Inserting the new data...


100%|██████████| 3/3 [00:16<00:00,  5.43s/it]

Updating the data_modified_on field...
